In [1]:
import pandas as pd

data = {
    '搜尋詞': ['淋浴水龍頭']*10,
    'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'tf-idf': [
        'JAVA 雙頭淋浴水龍頭 TLS414, 1組', 
        'MIRINAE 壁掛式淋浴水龍頭 MF-102, 1組', 
        'American Standard 美標 淋浴水龍頭 FB2689, 1個', 
        'DAELIM BATH 洗臉台淋浴水龍頭 DL-B1615, 1組', 
        'HANSSEM 淋浴方形水龍頭 HI-9030', 
        '高級水龍頭 黑色, 1個', 
        '高級水龍頭 藍色, 1個', 
        'YJJ 水龍頭過濾器3.0型水槽旋轉水龍頭龍頭套裝, 1組', 
        '水龍頭全浴美容洗髮水台水龍頭TPF-0105, 1套', 
        '我們的水早晨壁掛式單桿淋浴浴缸水龍頭, 1個'],
    'relevancy_1': [2, 2, 2, 2, 2, 0, 0, 0, 0, 1],
    'ner_relevancy_1': [0] * 10,
    'semantic_model': [
        '淋浴水龍頭, 1入',
        '淋浴水龍頭組 黑色, 1個', 
        '趨勢淋浴豪華浴室水龍頭鍍鉻光澤 RA-600, 1個', 
        '藍水浴室壁掛式淋浴龍頭 2-600, 1個', 
        'Jeilbas 鍍鉻鍍金浴室淋浴龍頭套組, 1套', 
        '水龍頭龍頭兩孔頂級水槽管道淋浴龍頭EINS-3033, 1個', 
        '水壓上升節水淋浴噴頭銀色, 1個', 
        '淋浴龍頭 T-02, 1個', 
        '壁掛式浴室淋浴龍頭金屬 OV3-600, 1個', 
        '淋浴龍頭 GK-102, 1個'],
    'relevancy_2': [2, 2, 1, 2, 2, 2, 2, 2, 2, 2],
    'ner_relevancy_2': [0] * 10
}

df = pd.DataFrame(data)
df

,搜尋詞,Rank,tf-idf,relevancy_1,ner_relevancy_1,semantic_model,relevancy_2,ner_relevancy_2
0,淋浴水龍頭,1,"JAVA 雙頭淋浴水龍頭 TLS414, 1組",2,0,"淋浴水龍頭, 1入",2,0
1,淋浴水龍頭,2,"MIRINAE 壁掛式淋浴水龍頭 MF-102, 1組",2,0,"淋浴水龍頭組 黑色, 1個",2,0
2,淋浴水龍頭,3,"American Standard 美標 淋浴水龍頭 FB2689, 1個",2,0,"趨勢淋浴豪華浴室水龍頭鍍鉻光澤 RA-600, 1個",1,0
3,淋浴水龍頭,4,"DAELIM BATH 洗臉台淋浴水龍頭 DL-B1615, 1組",2,0,"藍水浴室壁掛式淋浴龍頭 2-600, 1個",2,0
4,淋浴水龍頭,5,HANSSEM 淋浴方形水龍頭 HI-9030,2,0,"Jeilbas 鍍鉻鍍金浴室淋浴龍頭套組, 1套",2,0
5,淋浴水龍頭,6,"高級水龍頭 黑色, 1個",0,0,"水龍頭龍頭兩孔頂級水槽管道淋浴龍頭EINS-3033, 1個",2,0
6,淋浴水龍頭,7,"高級水龍頭 藍色, 1個",0,0,"水壓上升節水淋浴噴頭銀色, 1個",2,0
7,淋浴水龍頭,8,"YJJ 水龍頭過濾器3.0型水槽旋轉水龍頭龍頭套裝, 1組",0,0,"淋浴龍頭 T-02, 1個",2,0
8,淋浴水龍頭,9,"水龍頭全浴美容洗髮水台水龍頭TPF-0105, 1套",0,0,"壁掛式浴室淋浴龍頭金屬 OV3-600, 1個",2,0
9,淋浴水龍頭,10,"我們的水早晨壁掛式單桿淋浴浴缸水龍頭, 1個",1,0,"淋浴龍頭 GK-102, 1個",2,0


In [2]:
from inference_result import load_model, inference, process_result
import torch, src.config as config

# all_attribute = ['品牌', '名稱', '產品', '產品序號', '顏色', '材質', '對象與族群', '適用物體、事件與場所', 
#                      '特殊主題', '形狀', '圖案', '尺寸', '重量', '容量', '包裝組合', '功能與規格']

# set device
config.string_device = 'cuda' if torch.cuda.is_available() else 'cpu'
config.device = torch.device(config.string_device)

# load model
model, tokenizer = load_model("clw8998/Product-Name-NER-model", device=config.device)

def process_inference_tags(model, tokenizer, inference_data, attribute):
    result = inference(model, tokenizer, inference_data, attribute, batch_size=32)
    result_dict = process_result(result)
    all_tags_dict = result_dict[inference_data[0].lower()]
    tags = [item[0] for value in all_tags_dict.values() if value for item in value]
    return tags

def ner_relevancy(df):
    for index, row in df.iterrows():
        # 處理 '搜尋詞' 的 tags
        query_tags = process_inference_tags(model, tokenizer, [row['搜尋詞']], ['品牌', '產品', '功能與規格'])

        # 處理 'tf-idf' 的 tags
        tfidf_tags = process_inference_tags(model, tokenizer, [row['tf-idf']], ['品牌', '產品', '功能與規格'])

        # 處理 'semantic_model' 的 tags
        semantic_tags = process_inference_tags(model, tokenizer, [row['semantic_model']], ['品牌', '產品', '功能與規格'])

        if len(set(query_tags) & set(tfidf_tags)) >= 2:
            df.loc[index, 'ner_relevancy_1'] = 2
        elif len(set(query_tags) & set(tfidf_tags)) >= 1:
            df.loc[index, 'ner_relevancy_1'] = 1
        else:
            df.loc[index, 'ner_relevancy_1'] = 0

        if len(set(query_tags) & set(semantic_tags)) >= 2:
            df.loc[index, 'ner_relevancy_2'] = 2
        elif len(set(query_tags) & set(semantic_tags)) >= 1:
            df.loc[index, 'ner_relevancy_2'] = 1
        else:
            df.loc[index, 'ner_relevancy_2'] = 0

    return df

c:\Users\clw20\anaconda3\envs\cl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = ner_relevancy(df)
df

100%|██████████| 1/1 [00:00<00:00,  3.62it/s]


,搜尋詞,Rank,tf-idf,relevancy_1,ner_relevancy_1,semantic_model,relevancy_2,ner_relevancy_2
0,淋浴水龍頭,1,"JAVA 雙頭淋浴水龍頭 TLS414, 1組",2,2,"淋浴水龍頭, 1入",2,2
1,淋浴水龍頭,2,"MIRINAE 壁掛式淋浴水龍頭 MF-102, 1組",2,2,"淋浴水龍頭組 黑色, 1個",2,2
2,淋浴水龍頭,3,"American Standard 美標 淋浴水龍頭 FB2689, 1個",2,2,"趨勢淋浴豪華浴室水龍頭鍍鉻光澤 RA-600, 1個",1,1
3,淋浴水龍頭,4,"DAELIM BATH 洗臉台淋浴水龍頭 DL-B1615, 1組",2,2,"藍水浴室壁掛式淋浴龍頭 2-600, 1個",2,1
4,淋浴水龍頭,5,HANSSEM 淋浴方形水龍頭 HI-9030,2,2,"Jeilbas 鍍鉻鍍金浴室淋浴龍頭套組, 1套",2,1
5,淋浴水龍頭,6,"高級水龍頭 黑色, 1個",0,1,"水龍頭龍頭兩孔頂級水槽管道淋浴龍頭EINS-3033, 1個",2,1
6,淋浴水龍頭,7,"高級水龍頭 藍色, 1個",0,1,"水壓上升節水淋浴噴頭銀色, 1個",2,1
7,淋浴水龍頭,8,"YJJ 水龍頭過濾器3.0型水槽旋轉水龍頭龍頭套裝, 1組",0,0,"淋浴龍頭 T-02, 1個",2,1
8,淋浴水龍頭,9,"水龍頭全浴美容洗髮水台水龍頭TPF-0105, 1套",0,1,"壁掛式浴室淋浴龍頭金屬 OV3-600, 1個",2,1
9,淋浴水龍頭,10,"我們的水早晨壁掛式單桿淋浴浴缸水龍頭, 1個",1,2,"淋浴龍頭 GK-102, 1個",2,1


In [4]:
df.to_csv('M11207321-NER-relevancy.csv', index=False, encoding='utf-8-sig')